# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [121]:
%matplotlib ipympl 

In [122]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [123]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Bengkulu,ID,73.45,light rain,-3.8004,102.2655,Dena Hotel
1,1,Norman Wells,CA,-9.36,overcast clouds,65.2820,-126.8329,Yamouri Inn
2,2,Pitimbu,BR,79.39,scattered clouds,-7.4706,-34.8086,Pousada Brisa Mar
3,3,Sao Filipe,CV,75.51,overcast clouds,14.8961,-24.4956,Bela Vista
4,5,Kapaa,US,80.92,clear sky,22.0752,-159.3190,Pono Kai Resort


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [124]:
# Configure the map plot

map_plot = vacation_df.hvplot.points(
    "Lng", "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [125]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City'] == 'Nago'].copy()
vacation_end =  vacation_df.loc[vacation_df['City'] == 'Kaseda'].copy()
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Katsuura'].copy()
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Shirakawa'].copy()
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Kaseda'].copy()

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [126]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = vacation_df.reset_index(drop=True)
itinerary_df = pd.concat([vacation_start,vacation_end ,vacation_stop1,vacation_stop2 , vacation_stop3])


# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
87,119,Nago,JP,65.25,overcast clouds,26.5881,127.9761,白浜
228,345,Kaseda,JP,48.07,scattered clouds,31.4167,130.3167,加世田ホテルよしや
250,382,Katsuura,JP,42.75,overcast clouds,35.1333,140.3000,潮騒の宿
296,448,Shirakawa,JP,32.38,broken clouds,37.1223,140.2376,ホテルルートイン 新白河駅東
228,345,Kaseda,JP,48.07,scattered clouds,31.4167,130.3167,加世田ホテルよしや


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [127]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lat', 'Lng']]


# Display sample data
waypoints_df


,Lat,Lng
87,26.5881,127.9761
228,31.4167,130.3167
250,35.1333,140.3000
296,37.1223,140.2376
228,31.4167,130.3167


## Use GeoViews to create map that shows the four cities in the itinerary

In [128]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 600,
    frame_height = 500,
    size = "Max Temp",
    scale = .5,
    color = "City"
)

In [129]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [130]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [131]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '8f2ec9295ebe443baee9b73774b4fef8',
 'waypoints': '26.5881,127.9761|31.4167,130.3167|35.1333,140.3|37.1223,140.2376|31.4167,130.3167'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [132]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [127.9761, 26.5881], 'original_index': 0},
     {'location': [130.3167, 31.4167], 'original_index': 1},
     {'location': [140.3, 35.1333], 'original_index': 2},
     {'location': [140.2376, 37.1223], 'original_index': 3},
     {'location': [130.3167, 31.4167], 'original_index': 4}],
    'units': 'metric',
    'distance': 4123624,
    'distance_units': 'meters',
    'time': 217885.191,
    'legs': [{'distance': 766783,
      'time': 84916.414,
      'steps': [{'from_index': 0,
        'to_index': 2,
        'distance': 186,
        'time': 26.821,
        'instruction': {'text': 'Drive southeast.'}},
       {'from_index': 2,
        'to_index': 12,
        'distance': 216,
        'time': 28.954,
        'instruction': {'text': 'Turn left.'}},
       {'from_index': 12,
        'to_index': 39,
        'distance': 1275,
        'time': 68.393,
        'instruction': {'text': 'Turn left ont

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [133]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates']
legs

[[[127.976585, 26.588972],
  [127.978195, 26.588256],
  [127.978244, 26.5883],
  [127.978167, 26.588335],
  [127.978145, 26.58836],
  [127.978145, 26.588395],
  [127.978179, 26.588453],
  [127.978409, 26.588699],
  [127.978661, 26.588963],
  [127.978886, 26.589212],
  [127.979239, 26.589587],
  [127.979399, 26.589722],
  [127.97944, 26.589757],
  [127.979281, 26.589857],
  [127.97883, 26.590126],
  [127.978595, 26.590258],
  [127.978337, 26.590387],
  [127.978002, 26.590544],
  [127.977761, 26.590654],
  [127.977477, 26.590783],
  [127.977262, 26.590872],
  [127.977097, 26.590936],
  [127.976895, 26.591013],
  [127.976137, 26.591287],
  [127.976009, 26.591334],
  [127.97589, 26.591378],
  [127.973391, 26.592306],
  [127.973263, 26.592355],
  [127.970555, 26.59338],
  [127.970438, 26.59342],
  [127.969773, 26.593661],
  [127.969536, 26.59374],
  [127.969319, 26.593801],
  [127.96911, 26.593843],
  [127.9689, 26.593869],
  [127.96859, 26.593888],
  [127.968225, 26.593904],
  [127.967979,

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [134]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for index in legs:
    longitude = row["Lng"]
    latitude = row['Lat']
    
# longitude
latitude

31.4167

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [135]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df['longitude'] = longitude
route_df['latitude'] = latitude
# Display sample data
route_df.head()

,longitude,latitude


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [136]:
# Configure the route path by using the GeoViews' Path function
route_path = # YOUR CODE HERE

SyntaxError: invalid syntax (2377073273.py, line 2)

In [ ]:
# Display a composed plot by using the route_map and route_path objects
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]